### Lógica para leer el archivo de metadatos y pasarlo a formato CSV

En este caso solo nos importan los siguientes atributos
- biosample
- sra
- geographic location
- longitude & latitude


In [10]:
import re
import pandas as pd

# Leer todo el archivo
with open("../Data/honey_wgs-biosamples.txt", "r", encoding="utf-8") as f:
    texto = f.read()

# Dividir por muestras según patrón: número seguido de ":"
bloques = re.split(r'\n\d+:\s', "\n" + texto)  
# El primer elemento puede estar vacío porque pusimos "\n" al inicio
bloques = [b for b in bloques if b.strip() != ""]

datos = []

for bloque in bloques:
    # Extraer BioSample
    bio_match = re.search(r'BioSample:\s*([^;]+)', bloque)
    biosample = bio_match.group(1).strip() if bio_match else None
    
    # Extraer SRA
    sra_match = re.search(r'SRA:\s*([^\n;]+)', bloque)
    sra = sra_match.group(1).strip() if sra_match else None
    
    
    # Otros campos: Accession, Geographic location, Latitude y Longitude, Organism, Isolation source, Collection date
    accession_match = re.search(r'Accession:\s*([^\s]+)', bloque)
    accession = accession_match.group(1).strip() if accession_match else None
    
    geo_match = re.search(r'/geographic location="([^"]+)"', bloque)
    geographic_location = geo_match.group(1).strip() if geo_match else None
    
    latlong_match = re.search(r'/latitude and longitude="([^"]+)"', bloque)
    latlong = latlong_match.group(1).strip() if latlong_match else None
    lat = None
    lon = None
    if latlong and latlong.lower() != "not applicable":
        # Intentamos dividir por espacio, si está en formato "lat N lon W"
        parts = latlong.split()
        if len(parts) >= 3:
            # print(parts)
            lat = float(parts[0])
            lon = float(parts[2])
            if parts[1] == 'S':
                lat = lat * -1
            if parts[3] == 'W':
                lon = lon * -1
        else:
            pass
            # print(parts)
    
    # Guardar en lista
    datos.append({
        "BioSample": biosample,
        "SRA": sra,
        "Geographic location": geographic_location,
        "Latitude": lat,
        "Longitude": lon,
    })

df = pd.DataFrame(datos)
df = df[df['Geographic location'] != 'not applicable']



# Guardar
df.to_csv("metadatos.csv", index=False)
